In [11]:
%pip install natsort

  Using cached natsort-8.2.0-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install unidecode

  Using cached Unidecode-1.3.4-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
# import natsort
import unidecode
import logging
import sys
import os
import pandas  as pd
from io import StringIO

## include  type conversion

In [16]:
csv_string = \
"""
video_codec,audio_codec,audio_channels,is_avc,path_file,format_name
h264,aac,2,1,/path/to/video1.mp4,mov
h264,mp3,1,1,/path/to/video2.mkv,matroska
h264,aac,2,1,/path/to/video3.mp4,mov
vp9,opus,2,1,/path/to/video4.mkv,"mov,mp4,m4a,3gp,3g2,mj2"
h264,aac,5,1,/path/to/video5.mp4,mov
h265,aac,2,0,/path/to/video6.mkv,matroska
vp9,mp3,1,0,/path/to/video7.mkv,matroska
h264,aac,2,1,/path/to/video8.mp4,"mov,mp4,m4a,3gp,3g2,mj2"
h265,opus,2,1,/path/to/video9.mkv,"mov,mp4,m4a,3gp,3g2,mj2"
h264,aac,2,1,/path/to/video10.mp4,"mov,mp4,m4a,3gp,3g2,mj2"
"""

df_raw = pd.read_csv(StringIO(csv_string))
df_raw

,video_codec,audio_codec,audio_channels,is_avc,path_file,format_name
0,h264,aac,2,1,/path/to/video1.mp4,mov
1,h264,mp3,1,1,/path/to/video2.mkv,matroska
2,h264,aac,2,1,/path/to/video3.mp4,mov
3,vp9,opus,2,1,/path/to/video4.mkv,"mov,mp4,m4a,3gp,3g2,mj2"
4,h264,aac,5,1,/path/to/video5.mp4,mov
5,h265,aac,2,0,/path/to/video6.mkv,matroska
6,vp9,mp3,1,0,/path/to/video7.mkv,matroska
7,h264,aac,2,1,/path/to/video8.mp4,"mov,mp4,m4a,3gp,3g2,mj2"
8,h265,opus,2,1,/path/to/video9.mkv,"mov,mp4,m4a,3gp,3g2,mj2"
9,h264,aac,2,1,/path/to/video10.mp4,"mov,mp4,m4a,3gp,3g2,mj2"


In [18]:
df = df_raw.copy()
mask_cv_ok = df["video_codec"].isin(["h264"])
mask_ca_ok = df["audio_codec"].isin(["aac"])
mask_ac_ok = df["audio_channels"] <= 2
mask_isavc = df["is_avc"].isin([1])
mask_mp4 = df["path_file"].apply(
    lambda x: Path(x).suffix.lower() == ".mp4"
)
mask_format = df["format_name"] == "mov,mp4,m4a,3gp,3g2,mj2"

stream_video_ok = mask_cv_ok & mask_isavc
audio_stream_ok = mask_ca_ok & mask_ac_ok
container_ok = mask_mp4 & mask_format

df["type_conversion"] = ""

mask_1_not_needed = stream_video_ok & audio_stream_ok & container_ok
df.loc[mask_1_not_needed, "type_conversion"] = "1_not_needed"

mask_2_container = (stream_video_ok & audio_stream_ok) & ~container_ok
df.loc[mask_2_container, "type_conversion"] = "2_container"

mask_3_only_audio = stream_video_ok & ~audio_stream_ok
df.loc[mask_3_only_audio, "type_conversion"] = "3_only_audio"

mask_4_only_video = ~stream_video_ok & audio_stream_ok
df.loc[mask_4_only_video, "type_conversion"] = "4_only_video"

mask_5_total_conv = ~stream_video_ok & ~audio_stream_ok
df.loc[mask_5_total_conv, "type_conversion"] = "5_total_conv"
df

,video_codec,audio_codec,audio_channels,is_avc,path_file,format_name,type_conversion
0,h264,aac,2,1,/path/to/video1.mp4,mov,2_container
1,h264,mp3,1,1,/path/to/video2.mkv,matroska,3_only_audio
2,h264,aac,2,1,/path/to/video3.mp4,mov,2_container
3,vp9,opus,2,1,/path/to/video4.mkv,"mov,mp4,m4a,3gp,3g2,mj2",5_total_conv
4,h264,aac,5,1,/path/to/video5.mp4,mov,3_only_audio
5,h265,aac,2,0,/path/to/video6.mkv,matroska,4_only_video
6,vp9,mp3,1,0,/path/to/video7.mkv,matroska,5_total_conv
7,h264,aac,2,1,/path/to/video8.mp4,"mov,mp4,m4a,3gp,3g2,mj2",1_not_needed
8,h265,opus,2,1,/path/to/video9.mkv,"mov,mp4,m4a,3gp,3g2,mj2",5_total_conv
9,h264,aac,2,1,/path/to/video10.mp4,"mov,mp4,m4a,3gp,3g2,mj2",1_not_needed


In [19]:
for conv_type in df['type_conversion'].unique():
    print(conv_type)
    display(df.loc[df['type_conversion'].isin([conv_type])])
    print('---')

2_container


,video_codec,audio_codec,audio_channels,is_avc,path_file,format_name,type_conversion
0,h264,aac,2,1,/path/to/video1.mp4,mov,2_container
2,h264,aac,2,1,/path/to/video3.mp4,mov,2_container


---
3_only_audio


,video_codec,audio_codec,audio_channels,is_avc,path_file,format_name,type_conversion
1,h264,mp3,1,1,/path/to/video2.mkv,matroska,3_only_audio
4,h264,aac,5,1,/path/to/video5.mp4,mov,3_only_audio


---
5_total_conv


...

---
4_only_video


,video_codec,audio_codec,audio_channels,is_avc,path_file,format_name,type_conversion
5,h265,aac,2,0,/path/to/video6.mkv,matroska,4_only_video


---
1_not_needed


,video_codec,audio_codec,audio_channels,is_avc,path_file,format_name,type_conversion
7,h264,aac,2,1,/path/to/video8.mp4,"mov,mp4,m4a,3gp,3g2,mj2",1_not_needed
9,h264,aac,2,1,/path/to/video10.mp4,"mov,mp4,m4a,3gp,3g2,mj2",1_not_needed


---


## outros

In [6]:
folder_path = r'D:\ztest_split'
folder_path = Path(folder_path)

In [24]:
def get_file_path_converted_old(path_origin):
    " converte caminho absoluto de um arquivo de vídeo, para seu equivalente em mp4"
    name_origin = os.path.basename(path_origin)
    folder_origin = os.path.dirname(path_origin)
    name_wo_ext = os.path.splitext(name_origin)[0]
    name_converted = name_wo_ext + ".mp4"
    file_path_converted = os.path.join(folder_origin, name_converted)
    return file_path_converted

path_origin = r'D:\ztest_split\meuvideo.ts'
get_file_path_converted_old(path_origin)

'D:\\ztest_split\\meuvideo.mp4'

In [27]:
def get_file_path_converted(path_origin: Path):
    " converte caminho absoluto de um arquivo de vídeo, para seu equivalente em mp4"
    file_path_c = path_origin.parent / (path_origin.stem + '.mp4')
    return file_path_c

path_origin = Path(r'D:\ztest_split\meuvideo.ts')
get_file_path_converted(path_origin)

WindowsPath('D:/ztest_split/meuvideo.mp4')

In [7]:
list_dir_name = [path for path in folder_path.iterdir() if path.is_dir()]
list_dir_name

[WindowsPath('D:/ztest_split/1'),
 WindowsPath('D:/ztest_split/2'),
 WindowsPath('D:/ztest_split/3')]

In [10]:
import vidqa

In [12]:
import importlib

In [13]:
importlib.reload(vidqa)

<module 'vidqa' from 'd:\\git\\vidqa\\.venv\\lib\\site-packages\\vidqa\\__init__.py'>

In [8]:
vidqa.

AttributeError: module 'vidqa' has no attribute 'get_all_file_path'

In [50]:

utils_folder = Path('.').absolute().parent / 'vidqa'
sys.path.append(str(utils_folder))
import utils

In [56]:
folder_path = Path(r'D:\git\vidqa\vidqa\tests\lista_ordem_bug')

In [57]:
all_file_path = utils.get_all_file_path(folder_path)

D:\git\vidqa\vidqa\tests\lista_ordem_bug\Aula-003-Revisada.pdf
D:\git\vidqa\vidqa\tests\lista_ordem_bug\Aulá-002-Revisada.pdf


In [ ]:
list_errorpath = all_file_path['errors']
list_filepath = all_file_path['content']

In [11]:
def get_all_file_path(folder_path: Path, sort=True) -> dict[str, list[Path]]:

    """Returns List of all file paths inside a folder, recursively.
    Option to Sort naturally.

    Args:
    -----
        folder_path (Path): folder path
        sort (bool, optional): Return classified. Defaults to True.

    Returns:
    --------
        dict[str, list]: keys: ['content', 'errors']. values: list[Path]
    """

    def iter_folder(sub_folder):

        list_error = []
        list_file_path = []
        for x in folder_path.iterdir():
            if not x.exists():
                logging.error("path_too_long: %s", x)
                list_error.append(x)
                continue
            if x.is_dir():
                iter_folder(x)
            else:
                list_file_path.append(x)
        return list_file_path, list_error
    
    if not folder_path.exists():
        logging.error("Folder not exists: %s", folder_path)
        raise FileNotFoundError(f"Folder not exists: {folder_path}")

    list_file_path, list_error = iter_folder(folder_path)
    
    if sort:
        list_file_path = natsort.natsorted(
            list_file_path, lambda x: unidecode.unidecode(str(x).lower())
        )
        list_error = natsort.natsorted(
            list_error, lambda x: unidecode.unidecode(str(x).lower())
        )
    
    return {"content": list_file_path, "errors": list_error}

logging.basicConfig(level=logging.INFO)
folder_path = Path(r'D:\git\vidqa\vidqa\tests\lista_ordem_bug')

get_all_file_path(folder_path)


{'content': [WindowsPath('D:/git/vidqa/vidqa/tests/lista_ordem_bug/Aulá-002-Revisada.pdf'),
  WindowsPath('D:/git/vidqa/vidqa/tests/lista_ordem_bug/Aula-003-Revisada.pdf')],
 'errors': []}

In [3]:
logging.basicConfig(level=logging.INFO)
stringa = 'aaa'
logging.info('path_too_long: %s', stringa)

INFO:root:path_too_long: aaa
